In [11]:
import pandas as pd
from ast import literal_eval

# Cargar el archivo CSV
user_items = pd.read_csv('user_items.csv', converters={'items': literal_eval})
data_games = pd.read_csv('steam_games.csv')

In [3]:
print(user_items.columns)

Index(['user_id', 'items_cleaned'], dtype='object')


In [8]:
import pandas as pd
from ast import literal_eval

# Cargar el archivo CSV
user_items = pd.read_csv('user_items.csv', converters={'items_cleaned': literal_eval})

# Crear una lista para almacenar los DataFrames
dfs = []

# Iterar sobre el DataFrame 'user_items'
for index, row in user_items.iterrows():
    user_id = row['user_id']
    items_cleaned = row['items_cleaned']
    
    # Crear una lista para almacenar las filas de cada usuario
    rows = []

    # Iterar sobre los items de cada usuario
    for item in items_cleaned:
        item_id = item['item_id']
        rows.append({'user_id': user_id, 'item_id': item_id})

    # Agregar la lista de filas al DataFrame
    temp_df = pd.DataFrame(rows)

    # Agregar el DataFrame temporal a la lista
    dfs.append(temp_df)

# Concatenar todos los DataFrames en uno solo
user_item_mapping = pd.concat(dfs, ignore_index=True)

# Guardar el resultado en un archivo CSV
user_item_mapping.to_csv('user_item_mapping.csv', index=False)


In [9]:
user_item_mapping

,user_id,item_id
0,76561197970982479,10
1,76561197970982479,20
2,76561197970982479,30
3,76561197970982479,40
4,76561197970982479,50
...,...,...
5153204,76561198329548331,346330
5153205,76561198329548331,373330
5153206,76561198329548331,388490
5153207,76561198329548331,521570


In [12]:
data_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30067 entries, 0 to 30066
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     28833 non-null  object 
 1   app_name   30067 non-null  object 
 2   title      30067 non-null  object 
 3   tags       29906 non-null  object 
 4   specs      29398 non-null  object 
 5   price      27242 non-null  float64
 6   id         30066 non-null  float64
 7   developer  28818 non-null  object 
 8   year       29782 non-null  float64
dtypes: float64(3), object(6)
memory usage: 2.1+ MB


In [21]:
# Agrupar por 'id' y calcular la suma de 'price' para cada grupo
sum_price_by_id = data_games.groupby('id')['price'].sum().reset_index()

# Seleccionar solo las columnas 'id' y 'price_total'
result_df = sum_price_by_id[['id', 'price']]

# Renombrar la columna 'price' a 'total_price'
result_df = result_df.rename(columns={'price': 'total_price'})

# Guardar el resultado en un nuevo archivo CSV
result_df.to_csv('steam_games_with_total_price.csv', index=False)


In [14]:
user_reviews = pd.read_csv('user_reviews.csv', converters={'items': literal_eval})

In [15]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [19]:
# Inicializar una lista para almacenar los resultados
result_list = []

# Iterar sobre cada fila del DataFrame original
for index, row in user_reviews.iterrows():
    user_id = row['user_id']
    reviews = row['reviews']

    # Inicializar contadores
    total_reviews = 0
    true_reviews = 0

    # Iterar sobre cada review del usuario
    for review in reviews:
        # Obtener el valor de 'recommend'
        recommend_value = review.get('recommend', False)

        # Actualizar contadores
        total_reviews += 1
        if recommend_value:
            true_reviews += 1

    # Calcular el porcentaje de 'True' sobre el total de recomendaciones
    if total_reviews > 0:
        recommend_percentage = (true_reviews / total_reviews) * 100
    else:
        recommend_percentage = 0

    # Agregar resultados a la lista
    result_list.append({
        'user_id': user_id,
        'recommend_percentage': recommend_percentage
    })

# Convertir la lista en un DataFrame
result_df = pd.DataFrame(result_list)

# Guardar el resultado en un nuevo archivo CSV
result_df.to_csv('user_recommend_percentage.csv', index=False)


In [20]:
result_df

,user_id,recommend_percentage
0,76561197970982479,100.0
1,js41637,100.0
2,evcentric,100.0
3,doctr,100.0
4,maplemage,100.0
...,...,...
25794,76561198306599751,100.0
25795,Ghoustik,100.0
25796,76561198310819422,100.0
25797,76561198312638244,100.0


In [22]:
user_item_mapping = pd.read_csv('user_item_mapping.csv')
steam_games_with_total_price = pd.read_csv('steam_games_with_total_price.csv')
user_recommend_percentage = pd.read_csv('user_recommend_percentage.csv')

# Contar la cantidad de registros por 'user_id'
count_item_df = user_item_mapping.groupby('user_id').size().reset_index(name='count_item')


In [23]:
# Fusionar user_item_mapping con steam_games_with_total_price
merged_df = pd.merge(user_item_mapping, steam_games_with_total_price, left_on='item_id', right_on='id')

# Agrupar por 'user_id' y calcular la suma de 'total_price'
sum_price_df = merged_df.groupby('user_id')['total_price'].sum().reset_index(name='sum_price')


In [24]:
# Fusionar count_item_df y sum_price_df
result_df = pd.merge(count_item_df, sum_price_df, on='user_id', how='left')

# Fusionar result_df con user_recommend_percentage
result_df = pd.merge(result_df, user_recommend_percentage, on='user_id', how='left')


In [25]:
result_df.to_csv('def_userdata.csv', index=False)


In [33]:
import pandas as pd

def userdata(user_id):
    # Leer el archivo result_file.csv
    result_df = pd.read_csv('def_userdata.csv')

    # Filtrar el DataFrame para el usuario específico
    user_data = result_df[result_df['user_id'] == user_id]

    # Verificar si el usuario existe
    if user_data.empty:
        return {"error": "Usuario no encontrado"}

    # Obtener los valores requeridos
    dinero_gastado = user_data['sum_price'].values[0]
    recomendacion_pct = user_data['recommend_percentage'].values[0]
    cantidad_items = user_data['count_item'].values[0]

    # Formatear el porcentaje de recomendación
    recomendacion_pct_str = f"{recomendacion_pct * 1:.2f}%"

    # Construir el diccionario de resultados
    result_dict = {
        "Usuario X": user_id,
        "Dinero gastado": f"{dinero_gastado:.2f} USD",
        "% de recomendación": recomendacion_pct_str,
        "cantidad de items": cantidad_items
    }

    return result_dict



In [34]:
usuario_resultado = userdata('76561197970982479')
print(usuario_resultado)

{'Usuario X': '76561197970982479', 'Dinero gastado': '3419.32 USD', '% de recomendación': '100.00%', 'cantidad de items': 277}
